# Trabajo Final: IA Gamificada para Niños con TDAH: Aprender Jugando con Prompt Engineering

## Resumen

Se propone el diseño de un sistema de generación de contenido educativo gamificado mediante prompt engineering, orientado a niños de entre 8 y 12 años con Trastorno por Déficit de Atención e Hiperactividad (TDAH).
El objetivo es transformar lecciones escolares en experiencias interactivas, con misiones, preguntas de opción múltiple y retroalimentación positiva. Se implementan prompts adaptados para la API de Gemini, garantizando la posibilidad de producir salidas estructuradas en JSON, lo que facilita el desarrollo de aplicaciones interactivas. Asimismo, se incluyen ejemplos de prompts para generar imágenes educativas que complementen las historias.

## Índice

1. Introducción
2. Objetivos
3. Metodología
4. Herramientas y Tecnologías
5. Viabilidad
6. Implementación

## 1. Introducción

El Trastorno por Déficit de Atención e Hiperactividad (TDAH) afecta la forma en que muchos niños aprenden y procesan la información. En particular, entre los 8 y 12 años, la concentración sostenida y la memorización de conceptos abstractos resultan muy difíciles.
En contextos educativos tradicionales, los niños con TDAH suelen perder la motivación rápidamente, lo que deriva en frustración, bajo rendimiento y una percepción negativa del estudio. Es un problema relevante porque impacta no solo en lo académico, sino también en la autoestima y en la integración social de los niños.

## 2. Objetivos

* Diseñar prompts capaces de transformar contenido educativo en narrativas gamificadas.
* Incorporar preguntas de opción múltiple con retroalimentación positiva inmediata.
* Definir un formato estructurado de salida (JSON) para facilitar la integración con aplicaciones.

## 3. Metodología

1. **Diseño de prompts:**
   * Texto → Texto: generar historias interactivas, preguntas y feedback en JSON.
   * Texto → Imagen: generar personajes, escenarios y recompensas visuales.
2. **Pruebas de salida:** validar que los modelos devuelvan JSON válido y narrativas adaptadas a la edad objetivo.
3. **Iteración y ajuste:** modificar los prompts para mejorar la calidad de la narrativa y la claridad de las preguntas.

## 4. Herramientas y Tecnologías

* Python como lenguaje de implementación.
* Google Gemini API para generación de texto y, opcionalmente, imágenes.
* Gestión de variables de entorno con .env para proteger la API Key.
* Bibliotecas: google-genai, dotenv.

## 5. Viabilidad:

* Técnica:
  * Totalmente viable, tanto Gemini como chatGPT pueden realizar una respuesta en formato json, solo requiere un mínimo de procesamiento de texto previo ya que lo devuelve dentro de un formato markdown.
* Económicamente: 
  * La generación de imágenes es más costosa, pero se puede realizar una demostración técnica utilizando generadores gratuitos como leonardo.ai o Gemini en su versión web.

## 6. Implementación

### Requerimientos

Para poder ejecutarlo debe obtener una free api key de Gemini y crear un .env en la carpeta raíz con el siguiente código

`API_KEY="Tu api key"`

Antes de correrlo debe ejecutar el siguiente comando en la carpeta raíz 

`pip install -r "requirements.txt"`

### Código

### Carga de la API KEY desde el archivo .env

In [5]:
from dotenv import load_dotenv
import os

load_dotenv(".env")

mi_api_key = os.getenv("API_KEY")
if not mi_api_key:
    raise ValueError("API_KEY environment variable is not set.")

### Ejecución del prompt con las palabras claves a ingresar

Se diseñaron prompts parametrizados que aceptan variables, para poder cambiar el tema de la asignatura, la ambientación de la misión, la edad del niño y el tono de respuesta de la IA.

También se pide que responda en formato json para facilitar la creación de un software de preguntas y respuestas interactivas.

El prompt de las imágenes también fue parametrizado para que se pueda cambiar el personaje, el estilo de la imagen, la edad del niño al cual está dirigido y la pregunta que se desea ilustrar para contextualizar la misma.

#### _Prompt para obtener preguntas estilo múltiple Choice:_

Convierte esta lección sobre {tema} en una misión interactiva estilo {ambientacion}. 
La historia debe estar dirigida a un niño de {edad} años con TDAH. 
Incluye preguntas de opción múltiple, respuestas con feedback positivo, y un tono {tono}. 

Devuelve la respuesta en JSON válido con el siguiente formato:
```json
{{
  "historia": "texto breve con la narrativa gamificada",
  "preguntas": [
    {{
      "pregunta": "string",
      "opciones": ["string", "string", "string"],
      "respuesta_correcta": "string",
      "feedback": {{
        "correcto": "string",
        "incorrecto": "string"
      }}
    }}
  ]
}}
```

#### _Prompt para generar imágenes:_

Genera un personaje caricaturesco de {personaje}, en estilo {estilo}, 
colorido y amigable, pensado para un niño de {edad} años. 
El personaje debe estar contextualizado en base a la siguiente pregunta educativa: 
'{pregunta}'.

Por ejemplo:

Genera un personaje caricaturesco de un dragón amistoso, en estilo cartoon, 
colorido y amigable, pensado para un niño de 12 años. 
El contexto del personaje debe estar contextualizado en base a la siguiente pregunta educativa: 
'{pregunta obtenida en el prompt de generación de multiple choice}'.

In [6]:
from google import genai
from google.genai import types

client = genai.Client(api_key=mi_api_key)

'''
Variables del prompt
'''
tema=input("Ingrese el tema de la lección (Ej.: Fracciones): ")
ambientacion=input("Ingrese la ambientación de la misión (Ej.: Exploradores espaciales donde los niños deben repartir provisiones en partes iguales entre planetas): ")
edad=int(input("Ingrese la edad del niño a quien está dirigido (Ej.: 12): "))
tono=input("Ingrese el tono en el cual va a responder la IA a la respuesta ingresada (Ej.: Motivador): ")

# PROMPT
prompt = f"""
Convierte esta lección sobre {tema} en una misión interactiva estilo {ambientacion}. 
La historia debe estar dirigida a un niño de {edad} años con TDAH. 
Incluye preguntas de opción múltiple, respuestas con feedback positivo, y un tono {tono}. 

Devuelve la respuesta en JSON válido con el siguiente formato:

{{
  "historia": "texto breve con la narrativa gamificada",
  "preguntas": [
    {{
      "pregunta": "string",
      "opciones": ["string", "string", "string"],
      "respuesta_correcta": "string",
      "feedback": {{
        "correcto": "string",
        "incorrecto": "string"
      }}
    }}
  ]
}}
"""

response = client.models.generate_content(
    model = "gemini-2.5-flash",
    contents = prompt,
    config = types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0) # Desabilita el "Pensar" para dar respuestas mas rápidas y disminuir el tiempo de ejecución.
    ),
)

print(response.text)

```json
{
  "historia": "¡Atención, recluta! Aquí Comandante Águila. ¡Prepárate para la misión más importante de tu vida! El tiempo se nos agota. Estamos en plena Segunda Guerra Mundial y un valiente soldado, el Cabo Ryan, se ha perdido detrás de las líneas enemigas. Su familia lo espera. Tu misión, si decides aceptarla, es usar tu ingenio y tus conocimientos para guiar a nuestro equipo a través de los eventos cruciales de la guerra y traer de vuelta a Ryan. ¿Listo para ser un héroe? ¡Vamos a ello!",
  "preguntas": [
    {
      "pregunta": "Nuestra primera parada es crucial. Antes de que Estados Unidos entrara de lleno en la guerra, un evento masivo lo catapultó. ¿Cuál de estos ataques marcó la entrada de EE. UU. en la Segunda Guerra Mundial?",
      "opciones": [
        "El desembarco de Normandía",
        "El ataque a Pearl Harbor",
        "La batalla de Stalingrado"
      ],
      "respuesta_correcta": "El ataque a Pearl Harbor",
      "feedback": {
        "correcto": "¡Increíb